In [ ]:
import random, time
import numpy as np

OPT_X = 3.17007851
def fitness(x):
    return 50.0 - 100.0*(x-OPT_X)**2

ROWS, COLS = 4, 4       # small 4x4 torus -> POP=16 (good for notebook)
POP = ROWS*COLS
GENS = 30
LB, UB = 2.0, 4.0
MUT_STD = 0.03
NEIGH = [(-1,0),(1,0),(0,-1),(0,1),(0,0)]

random.seed(1)
np.random.seed(1)

def idx_to_rc(i): return divmod(i, COLS)
def rc_to_idx(r,c): return (r % ROWS) * COLS + (c % COLS)
def neigh_idx(i):
    r,c = idx_to_rc(i)
    return [rc_to_idx(r+dr, c+dc) for dr,dc in NEIGH]

def tournament_select(vals, scores):
    # return best two parents (highest scores)
    order = np.argsort(scores)
    return vals[order[-1]], vals[order[-2]]

def crossover(a,b):
    alpha = random.random()
    return alpha*a + (1-alpha)*b

def mutate(x):
    return max(LB, min(UB, x + random.gauss(0, MUT_STD)))

def evolve_cell(i, pop, scores):
    ids = neigh_idx(i)
    vals = np.array([pop[j] for j in ids])
    scs  = np.array([scores[j] for j in ids])
    p1, p2 = tournament_select(vals, scs)
    child = crossover(p1, p2)
    return mutate(child)

# run cellular GA
pop = list(np.random.uniform(LB, UB, POP))
scores = np.array([fitness(p) for p in pop])
start = time.time()
for g in range(GENS):
    new_pop = [evolve_cell(i, pop, scores) for i in range(POP)]
    new_scores = np.array([fitness(p) for p in new_pop])
    pop, scores = new_pop, new_scores
    best_i = int(np.argmax(scores))
    best_x, best_s = pop[best_i], scores[best_i]
    if (g+1) % 5 == 0 or g == 0:
        print(f"gen {g+1}/{GENS}  best_thickness={best_x:.8f}  fitness={best_s:.6f}")
    if abs(best_x - OPT_X) < 1e-8:
        break

elapsed = time.time() - start
best_i = int(np.argmax(scores))
print("\nOptimized beam thickness :", f"{pop[best_i]:.8f}")
print("Fitness :", f"{scores[best_i]:.6f}")
print("Elapsed sec:", f"{elapsed:.3f}", "generations:", g+1)
